<a href="https://colab.research.google.com/github/sarthakvinayaka/NLP-Project/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie review Classifier

In [11]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [0]:
#Downloading imdb data set from tensorflow_datasets
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

In [13]:
#spliting data set as train and test
import numpy as np
train_data,test_data=imdb['train'],imdb['test']

print(train_data)

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>


In [0]:

training_sentences=[]
training_labels=[]
testing_sentences=[]
testing_labels=[]


In [24]:
#appending train and test data in our array
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

for i in range(1,5):
  print(training_sentences[i])  
   
  

b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'
b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets fo

In [0]:
#creating array of training label and testing label using numpy
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [18]:
print(np.shape(training_labels_final))
print(np.shape(training_sentences))

(50000,)
(50000,)


In [0]:
#breaking each sentence to word and then utokenize the word with number and creating a array of it
vocab_size=10000
embedding_dim=16
max_length=120
trunc_type='post'
oov_tok='<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

In [0]:
#building LSTM( special type of RNN ) Model using kearas and tensorflow
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
 
    

In [21]:

num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
1563/1563 [==============================] - 162s 104ms/step - loss: 0.3799 - accuracy: 0.8235 - val_loss: 0.3651 - val_accuracy: 0.8424
Epoch 2/10
1563/1563 [==============================] - 158s 101ms/step - loss: 0.2048 - accuracy: 0.9249 - val_loss: 0.4137 - val_accuracy: 0.8347
Epoch 3/10
1563/1563 [==============================] - 152s 97ms/step - loss: 0.1276 - accuracy: 0.9564 - val_loss: 0.6526 - val_accuracy: 0.8211
Epoch 4/10
1563/1563 [==============================] - 152s 97ms/step - loss: 0.0769 - accuracy: 0.9744 - val_loss: 0.6615 - val_accuracy: 0.8162
Epoch 5/10
1563/1563 [==============================] - 130s 83ms/step - loss: 0.0423 - accuracy: 0.9860 - val_loss: 0.8238 - val_accuracy: 0.8184
Epoch 6/10
1563/1563 [==============================] - 137s 88ms/step - loss: 0.0287 - accuracy: 0.9907 - val_loss: 0.8605 - val_accuracy: 0.8126
Epoch 7/10
1563/1563 [==============================] - 134s 86ms/step - loss: 0.0226 - accuracy: 0.9925 - val_loss:

In [23]:
sentence = "boring"
sequence = tokenizer.texts_to_sequences([sentence])[0]

sequence=pad_sequences([sequence],maxlen=120,padding='pre')
k=model.predict_classes(sequence,verbose=0)
if k==0:
   print("Negative Review")
else:
   print("Positive review")

Negative Review
